## 1. mac_addr csv 가져오기

In [0]:
# csv path
csv_path = "file:/Workspace/EIC_데이터엔지니어/databricks_jira/eic/1218/mac_addr.csv"

# spark dataframe 
df = (spark.read
      .format("csv")
      .option("header", True)           # 첫 행을 컬럼명으로 사용
      .option("inferSchema", True)      # 간단한 경우 자동 스키마 추론
      # .option("delimiter", ",")       # 구분자 기본은 ',' (필요 시 지정)
      # .option("encoding", "utf-8")    # 인코딩 필요 시 지정
      .load(csv_path)
     )
    
# 맥 값 전처리 
from pyspark.sql import functions as F
df = df.withColumn(
    "mac_addr_2",
    F.concat_ws(
        ":",
        F.substring("MAC_ADDRESS", 1, 2),
        F.substring("MAC_ADDRESS", 3, 2),
        F.substring("MAC_ADDRESS", 5, 2),
        F.substring("MAC_ADDRESS", 7, 2),
        F.substring("MAC_ADDRESS", 9, 2),
        F.substring("MAC_ADDRESS", 11, 2),
    )
)

# 맥 해쉬 처리
tv_salt = dbutils.secrets.get('admin', 'salt')

df = df.withColumn("mac_addr_hashed", 
                   F.when(
                       F.col("mac_addr_2").isNull() | (F.col("mac_addr_2") == ''), 
                       None)\
                    .otherwise(F.sha2(F.concat(F.col("mac_addr_2"), F.lit(tv_salt)), 256)))

display(df)


In [0]:
import pandas as pd

# sdf -> pdf 
pdf = df.toPandas()
pdf.columns = ['_'.join(col.split(' ')) for col in pdf.columns]
pdf = pdf[['Production_Date'
           , 'Model.Suffix'
           , 'SET_ID'
           , 'MAC_ADDRESS'
           , 'mac_addr_hashed']]
pdf = pdf.reset_index(drop=False)
display(pdf)

In [0]:
spark.createDataFrame(pdf[['index', 'mac_addr_hashed']]).createOrReplaceTempView("tmp_mac_addr")

## 2. activation_date - min(crt_date) 구하기

In [0]:
df_result_1 = spark.sql(f''' 
    
    select raw.mac_addr
           , min(raw.crt_date) as min_crt_date 
           , first(raw.Platform_code) as platform_code 
           , max(raw.last_chg_date) as max_last_chg_date
           , date_format(max(raw.last_chg_date), 'yyyy-MM') as last_chg_date_ym
    from   eic_data_ods.tlamp.activation_date raw
    inner join tmp_mac_addr tmp on raw.mac_addr = tmp.mac_addr_hashed
    group by mac_addr
''')
display(df_result_1)

In [0]:
df_result_1\
    .write.mode('overwrite')\
    .option("mergeSchema", "true")\
    .saveAsTable(f"sandbox.z_eunmi1_ko.temp_1218_1") # 중간 저장 

## 3. 노말로그 값 구하기
> 3-1) max_last_chg_date를 yyyy-MM 로 바꾸고, date_ym별 mac_addr 대한 마지막 normal_log 추출 (union 활용) <br/>
> 3-2) 단, max_last_chg_date < '2025-12' 이면, 데이터 없음으로 추출 (2년 보관 중)

In [0]:
list_date_ym = []
df_result_1 = spark.table("sandbox.z_eunmi1_ko.temp_1218_1")
for row in df_result_1.select("last_chg_date_ym").distinct().collect():
    list_date_ym.append(row.last_chg_date_ym)
print(tuple(list_date_ym))

In [0]:
df_result_2 = spark.sql(f'''
    select nl.mac_addr
           , nl.log_create_time
           , nl.DEVICE_NAME 
           , nl.X_Device_Product 
           , nl.X_Device_FW_Version
       --     , nl.ip_addr 
       --     , nl.accum_run_time
       --     , nl.context_name
       --     , nl.message_id
       --     , nl.normal_log
       --     , nl.REPORT_TIME
       --     , nl.PRODUCT_NUM 
       --     , nl.BUILD_ID 
       --     , nl.BUILD_NAME 
       --     , nl.RELEASE_NUMBER 
       --     , nl.X_Device_Type 
       --     , nl.X_Device_SDK_VERSION 
       --     , nl.X_Device_Model 
       --     , nl.X_Device_Eula 
       --     , nl.X_Device_Netcast_Platform_Version 
       --     , nl.X_Device_Brand 
       --     , nl.X_Device_Locale 
       --     , nl.X_Device_Eco_Info 
       --     , nl.X_Device_Publish_Flag 
       --     , nl.X_Device_Platform 
       --     , nl.X_Device_Language 
       --     , nl.X_Device_FCK 
       --     , nl.X_Device_Country 
       --     , nl.X_Device_Country_Group 
       --     , nl.X_Device_Sales_Model 
       --     , nl.date_ym
    from   eic_data_ods.tlamp.normal_log_webos25 nl
    inner join tmp_mac_addr mac on nl.mac_addr = mac.mac_addr_hashed
    where  nl.date_ym in {tuple(list_date_ym)}
      and  nl.X_device_country = 'TR'
''')

display(df_result_2)

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window


# Window 정의
window_spec = Window.partitionBy("mac_addr").orderBy(F.col("date_ym").desc(), 
                                                     F.col("log_create_time").desc())

# rank() 적용
df_result_2 = df_result_2.withColumn("rn", F.rank().over(window_spec))

# rn = 1 조건 필터링
df_result_3 = df_result_2.filter(F.col("rn") == 1).select("*").drop("rn")

display(df_result_3)


In [0]:
df_result_3\
    .write.mode('overwrite')\
    .option("mergeSchema", "true")\
    .saveAsTable(f"sandbox.z_eunmi1_ko.temp_1218_3")

#### 최종 결과

In [0]:
%sql 
select mac.index
       , res1.min_crt_date
       , res1.platform_code
       , res3.log_create_time
       , res3.DEVICE_NAME
       , res3.X_Device_Product
       , res3.X_Device_FW_Version
from tmp_mac_addr mac
left join sandbox.z_eunmi1_ko.temp_1218_1 res1 on mac.mac_addr_hashed = res1.mac_addr 
left join sandbox.z_eunmi1_ko.temp_1218_3 res3 on mac.mac_addr_hashed = res3.mac_addr
;

-- csv 다운 후, mac_addr 컬럼은 수기 삭제. 이후, 전달 